In [11]:
##This is the only cell that you should have to make edits to, enter in your desired input and output file paths and change what you deem to be an acceptable recovery range
#raw data upload file path
raw_filepath = r'C:\Users\jhsan\OneDrive\Desktop\ACF_Project\Raw_Data\PFAS_ACF_Batch1.txt'

#processed data output file path
processed_filepath =r'C:\Users\jhsan\OneDrive\Desktop\ACF_Project\Processed_Data\PFAS_ACF_Batch1_processed.xlsx'

#file path to write QCS0 data to 
qcs0_filepath = r'C:\Users\jhsan\OneDrive\Desktop\QCS0_area_values\QCS0_area_values.csv' 

#file path for IDL and IQL data
IDL_IQL_filepath = r'C:\Users\jhsan\OneDrive\Desktop\ACF_Project\ACF\IDL_IQL.csv'

#color-coding for recoveries table
in_range = 'background-color: green'
in_range_min_val = 0.6 
in_range_max_val = 1.4
out_range = 'background-color: red'
out_range_min_val = 0.4 
out_range_max_val = 1.8
question_range = 'background-color: yellow'

In [22]:
import os
import re
import tempfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
wb= Workbook()
ws=wb.active

#Load data file
df=pd.read_csv(raw_filepath, delimiter='\t', low_memory=False)

#remove Calibtration data
df1 = df[df['Sample Type'] != 'Standard']

In [13]:
#Filter out all but IDA area values for recovery calculation
selected_columns = ['Sample Name', 'Sample Index','Acquisition Date & Time','Component Name', 'Area']
area=df1[selected_columns]
area_ida = area[area['Component Name'].str.contains('IDA')].copy()
area_ida.loc[:,'Sample Name Date'] = area_ida['Sample Name'].astype(str) + "_" + area_ida['Acquisition Date & Time']

In [14]:
# Create pivot table with Sample name as the index, component name as the column headers, and area as the values
area_piv = area_ida.pivot_table(index=('Sample Name Date',), columns='Component Name', values='Area', aggfunc='first')

In [15]:
# Filter rows where 'Sample Name' contains 'QCS0'
qcs0_samples = area_piv.reset_index()
qcs0_samples = qcs0_samples[qcs0_samples['Sample Name Date'].str.contains('QCS0')]

# Append the filtered DataFrame to an existing CSV file
if os.path.exists(qcs0_filepath):
    # Load the existing data
    existing_data = pd.read_csv(qcs0_filepath)
    
    # Combine existing data with new data, avoiding duplicates
    combined_data = pd.concat([existing_data, qcs0_samples]).drop_duplicates(subset=['Sample Name Date'])
    
    # Write back to the CSV without writing headers again
    combined_data.to_csv(qcs0_filepath, index=False)
else:
    # If the file doesn't exist, write the data with headers
    qcs0_samples.to_csv(qcs0_filepath, index=False)

In [16]:
##Recoveries
#Average QCS0 from saved csv file
QCS0=pd.read_csv(qcs0_filepath, low_memory=False)

#QCS0 averages
avg_QCS0 = QCS0.mean().to_frame(name='Average').T

#Generate base of recovery table by copying the area pivot data
recovery= area_piv.copy()

# Recovery calculation (sample area/avg QCS0 area)
for index, row in area_piv.iterrows():
    for col in area_piv.columns:
        if area_piv[col].dtype in ['float64', 'int64']:
            recovery.at[index, col] = row[col] / avg_QCS0[col]

In [17]:
#color code recoveries
def color_map(val):
    if in_range_min_val <= val <= in_range_max_val:
        return in_range
    elif val < out_range_min_val or val > out_range_max_val:
        return out_range
    else:
        return question_range

    # Apply the style function to the entire DataFrame
styled_recovery = recovery.style.applymap(color_map)

In [18]:
##MDL
#Load concentration data
selected_columns = ['Sample Name', 'Component Name', 'Calculated Concentration']

#Isolate blank data
df2 = df1[df1['Sample Comment'].str.contains('Blank')]
df2 = df2[selected_columns]

#Remove any IDA or IPS values
df2 = df2[~df2['Component Name'].str.contains('IDA|IPS')]

#Create pivot table with Sample name as the index, component name as the column header, and concentration as the value
df2_piv = df2.pivot_table(index='Sample Name', columns='Component Name', values='Calculated Concentration', aggfunc='first')

#Isolate the process blank data
PB = df2_piv[df2_piv.index.str.contains('PB')]

#replace all <1point values with NaN values
PB= PB.replace("<1 points", np.nan)

#Change any non numeric valeus to numeric
PB=PB.apply(pd.to_numeric, errors='coerce')

#calculate the average PB value excluding NaN values
PB_avg = np.nanmean(PB, axis=0)
#create a row called PB_avg
PB.loc['PB_avg'] = PB_avg

#calculate the standard deviation
PB_stdev = PB.std(skipna=True)
PB.loc['PB_stdev'] = PB_stdev

#MDL calculation PB_avg + 3*PB_stdev
PB.loc['MDL']=np.nan_to_num(PB.loc['PB_avg'])+ 3*np.nan_to_num(PB.loc['PB_stdev'])
PB.loc['MDL']=PB.loc['MDL'].replace(0, np.nan)

#Load IDL_IQL file
IDL_IQL = pd.read_csv(IDL_IQL_filepath, index_col=0, low_memory=False)

#change all non numeric values to numeric
IDL_IQL = IDL_IQL.apply(pd.to_numeric, errors='coerce')
IQL = IDL_IQL.loc[['IQL']]  
IQL = IQL.apply(pd.to_numeric, errors='coerce')

#replace all NaN values in MDL with the IQL value
common_columns = PB.columns.intersection(IQL.columns)
PB.loc['MDL', common_columns] = PB.loc['MDL', common_columns].combine_first(IQL.squeeze())

C:\Users\jhsan\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice


In [19]:
#Load concentration data
selected_columns = ['Sample Name', 'Component Name', 'Calculated Concentration']
df3=df1[df1['Sample Comment'] != 'Blank'][selected_columns].copy()
df3 = df3[~df3['Component Name'].str.contains('IDA|IPS')]
df3 = df3.pivot_table(index='Sample Name', columns='Component Name', values='Calculated Concentration', aggfunc='first')
df3= df3.replace("<1 points", np.nan)

# Align the columns of PB with df3
common_columns = df3.columns.intersection(PB.columns)
PB_aligned = PB[common_columns]

# Replace values in df3 that are less than MDL with '<MDL'
mdl_df = pd.DataFrame('<MDL', index=df3.index, columns=common_columns)

# Convert MDL values to numeric to handle both numeric and string types
mdl_values = pd.to_numeric(PB_aligned.loc['MDL'], errors='coerce')

# Use np.where with numeric comparison
df3[common_columns] = np.where(df3[common_columns].apply(pd.to_numeric, errors='coerce') < mdl_values.values, mdl_df, df3[common_columns])
#change all NaN to <MDL
df3 = df3.fillna('<MDL')


In [24]:
selected_columns = ['Sample Name', 'Component Name', 'Actual Concentration','IS Actual Concentration','Area','IS Area']
df4=df[df['Sample Name'].str.contains('PFAS CS')].copy()
df4 = df4.fillna(0)
df4 = df4[~df4['Component Name'].str.contains('IDA|IPS|13C|d3|d5|TOF')]
df4['Concentration/IS Concentration']=df4['Actual Concentration']/df4['IS Actual Concentration']
df4['Area/IS Area']=df4['Area']/df4['IS Area']
df4
# Create a list of unique sample names
components = df4['Component Name'].unique()
n_components = len(components)
#create single plot
#fig, axes = plt.subplots(n_components, 1, sharex=True, figsize = (8, n_components * 3))
cal_dir = tempfile.mkdtemp()
# Function to sanitize file names
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "_", name)
image_paths = []
# Iterate over each component and create scatter plots with regression lines
for i, component in enumerate(components):
    component_data = df4[df4['Component Name'] == component]
    
    # Extract x and y values
    x = component_data['Concentration/IS Concentration'].values.reshape(-1, 1)
    y = component_data['Area/IS Area'].values
    
    # Perform linear regression
    model = LinearRegression()
    model.fit(x, y)
    y_pred = model.predict(x)
    r2 = r2_score(y, y_pred)
# Regression equation
    slope = model.coef_[0]
    intercept = model.intercept_
    equation = f'y = {slope:.2f}x + {intercept:.2f}'
    plt.figure(figsize = (8,6))
    # Plot with Seaborn
    sns.regplot(
       # ax=axes[i], 
        x=x.flatten(), 
        y=y, 
        scatter=True, 
        fit_reg=True,
        line_kws={"color": "red"},  # Color of the regression line
        scatter_kws={"s": 50, "alpha": 0.7},  # Customize scatter points
        ci=95
    )
    # Set the title with the component name
    plt.title(f'{component}')
    plt.xlabel('Concentration/IS Concentration')
    plt.ylabel('Area/IS Area')
    plt.text(0.05, 0.95, f'{equation}\n$R^2$ = {r2:.2f}', 
             transform=plt.gca().transAxes, 
             fontsize=10, 
             verticalalignment='top', 
             bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))
   
    # Sanitize the file name
    sanitized_component = sanitize_filename(component)
    image_path = os.path.join(cal_dir, f'{sanitized_component}.png')
    
    # Save the plot as an image
    plt.savefig(image_path)
    plt.close()
    
    image_paths.append(image_path)


In [26]:
# Write data to worksheets in same workbook
with pd.ExcelWriter(processed_filepath, engine = 'openpyxl') as writer:
    writer.book = wb
    writer.sheets = dict((ws.title, ws)for ws in wb.worksheets)
    styled_recovery.to_excel(writer, sheet_name = 'Recoveries')
    PB.to_excel(writer, sheet_name = 'MDL_Values')
    df3.to_excel(writer, sheet_name = 'Concentration_filtered_MDL')
    area_piv.to_excel(writer, sheet_name = "Area_Pivot")
    IDL_IQL.to_excel(writer, sheet_name = "IDL_IQL")
    df4.to_excel(writer, sheet_name = "Calibration Data")
    # Saved Calibration graphs
    sheet_name = 'Calibration Curves'
    if sheet_name not in writer.sheets:
        writer.book.create_sheet(sheet_name)
    worksheet = writer.book[sheet_name]
    
    # Insert all images into one sheet in a grid format
    row_offset = 1  # Start at the first row
    col_offset = 1  # Start at the first column
    images_per_row = 2  # Number of images per row

    for i, image_path in enumerate(image_paths):
        # Calculate the position for each image
        row_position = row_offset + (i // images_per_row) * 20  # Adjust the multiplier to control spacing
        col_position = col_offset + (i % images_per_row) * 10   # Adjust the multiplier to control spacing
        
        # Load the image
        img = Image(image_path)
        
        # Place the image at the calculated position
        cell_position = worksheet.cell(row=row_position, column=col_position).coordinate
        worksheet.add_image(img, cell_position)

    

